In [7]:
import math
import numpy as np
import pandas as pd
import tqdm
import random


ROOT = 'C:\\Users\\Indrani Rana\\OneDrive\\Desktop\\project\\dada_half_systems'
DATA = 'C:\\Users\\Indrani Rana\\OneDrive\\Desktop\\project\\dada_half_systems\\SOSData'


In [8]:
#--- IMPORT Dataset and Normalize ------------------------------------------------------+
def load_data(data_loc,n_exp):
    '''
        data_loc: location of the data
        n_exp: no of data
    '''
    SOS_data = []            # N-D array data
    dfs = dict()             # each experiment as dataframe
    exps = []                # name of experiments, ['Exp1.txt,Exp2.txt,...]
    for i in range(1,n_exp+1):
        exps.append('Exp'+str(i)+'.txt')

    for exp in sorted(exps):
        df = pd.read_csv(data_loc+'/'+exp,delimiter='\t') # read txt file
        df = df.set_index(df.columns[0])
        #print(list(df.index))
        global gene_names
        gene_names = list(df.index)
        dfs[exp]=df

        # convert to numpy array of [8 X 50] 
        # [8 --> operons, 50 --> time points(0,6,12,18,..)]

        df_numpy = df.to_numpy()
        SOS_data.append(df_numpy)

    SOS_data = np.stack(SOS_data,axis=0)

    
    return SOS_data #,dfs if dataframes are required for individual experiment



def data_normalize(data):
    '''
        data : shape of [4x8x50]
    '''
    data1 = []
    for i in range(0,data.shape[0]):
        normalizedData = (data[i]-np.min(data[i]))/(np.max(data[i])-np.min(data[i]))
        data1.append(normalizedData)
    data1 = np.stack(data1,axis=0)
    return data1



In [9]:
def pi(x_j_t,f_ij):
    '''
        x_j_t :     actual value of j-th gene at t-th time point [1 x N]
        f_ij :      extracted f_ij from P_g [1 x N]

    '''
    temp = []
    for j in range(0,len(x_j_t)):
        temp.append(pow(x_j_t[j],f_ij[j]))
    return np.sum(temp)

def obj(X_i, X_pred_i):
    '''
        X_i :       actual data points of i-th gene [1 x 50]
        X_pred_i :  predicted data points of i-th gene [1 x 50]

    '''
    T = len(X_i)
    sum = 0
    for t in range(0,T):
        sum = sum + (X_i[t] - X_pred_i[t])
    return sum/T


def fitness_particles(X,g,P_g):
    '''
        X :         actual data of genes with 50 time points [N x 50]
        g :         current gene 
        P_g :       particle postions of current gene
    '''

    # Extract number of genes 
    N = X.shape[0]
    # Extract number of time points
    tp = X.shape[1]
    # Extract population size
    ps = P_g.shape[1]
    #print(ps)

    # error of all particles
    err_g = np.zeros((ps))

    for i in range(0,ps):
        # Extract f_ij_g, which is a array of N elements [f_i1, f_i2, ...... f_iN] for each i-th particle
        # also extract del_i, sci_i, and meu_i out of P_g or i-th particle p_i_g

        f_ij = P_g[g,i,:N]      # [1 x N]
        del_i = P_g[g,i,N]      # [1]
        sci_i = P_g[g,i,N+1]    # [1]
        meu_i = P_g[g,i,N+2]    # [1]

        for t in range(2,tp):
            # diff. between the (t-1)-th predicted data point and actual data point of g-th gene
            d_i = X[g,t-1] - X_pred[g,t-1]

            # Calculate the predicted expression level x_pred_i(t) of g-th gene from x_i(t-1)
            X_pred[g,t] = dt * (del_i * pi(X[:,t-1],f_ij)) + (1 - dt * sci_i) * X[g,t-1] + meu_i * d_i
        
        # error of i-th particle of g-th gene

        err_i_g = obj(X[g],X_pred[g])

        err_g[i] = err_i_g
    
    return err_g


def update_solution(p_g,per_best,per_best_err,gl_best,min_idx,err,best_fit):
    ps = p_g.shape[0]

    for i in range(0,ps):
        #print(err)
        if err[i] < per_best_err[i]:
            per_best_err[i] = err[i]
            per_best[i] = p_g[i]
        if err[i] < best_fit:
            best_fit = err[i]
            gl_best = p_g[i]
            min_idx = i
    return per_best,per_best_err,gl_best,best_fit,min_idx

def gen_random_numbers_in_range(low, high, n):
    return random.sample(range(low, high), n)


 
    



def BAPSO(N,maxit,ps,X):
    '''
        N :         Number of Genes
        maxit :     total number of iterations
        ps :        population size
        X :         data of shape [8x50]      
    '''
    # initialize position vector(P_g)  = random and velocity vector(V_g) = zeros for all N genes
    # each p_i_g of P_g (for each gene) is defined as [f_i_1, f_i_2, ...... f_i_N, del_i, sci_i, meu_i], which is a size of (1 x N+3)
    # then, total size of P_g is (N x ps x N+3)

    P_g = np.random.rand(N,ps,N+3)
    V_g = np.zeros((N,ps,N+3))


    # making f_ij sparse
    for g in range(0,N):
        for pos in range(0,ps):
            P_g[g,pos,:N] = 0
            idx = gen_random_numbers_in_range(0,N,4)
            P_g[g,pos,idx[0]] = P_g[g,pos,idx[1]] = P_g[g,pos,idx[2]] = P_g[g,pos,idx[3]] =1 
    #print(P_g)



    # pred data point matrix [N x 50]
    global X_pred
    X_pred = np.zeros((X.shape),dtype=np.float32)

    # the del_t, which is 6 mins
    global dt 
    dt = 6 * 60 # in sec

    # error matrix of all particles of all genes
    global err
    err = np.zeros((N,ps))


    # all best solutions
    GB = []

    for g in range(0,N):
        err_g = fitness_particles(X,g,P_g)

        min_idx = np.argmin(err_g) # find the particle index with minimum value
        best_fit = err_g[min_idx]   # fitness of that index

        # initialize personal best solutions PB_g [1 x ps] and its fitness PE_g [1 x ps]
        PB_g = P_g[g] # position [ps x (N+3)]
        PE_g = err_g  # error    [1 x ps]

        # calculate the global best soluton gb_g = PB_g[min_idx]
        gb_g = PB_g[min_idx] # [1 x (N+3)]

        err[g] = err_g

        for it in tqdm.tqdm(range(1,maxit),desc="Gene {}".format(g+1)):

            inertia_comp = r0 * V_g[g]      # [ps x (N+3)]
            cognitive_comp = c1 * r[0] * (PB_g - P_g[g])     # the particle’s memory, causing it to tend to return to 
                                                                    # the regions of the search space in which it has experienced high individual fitness

            social_comp = c2 * r[1] * (gb_g - P_g[g])         # causes the particle to move to the best region the swarm has found so far

            next_v =  inertia_comp + cognitive_comp + social_comp

            next_p = P_g[g] + next_v
            #print(next_v.shape,V_g.shape,gb_g.shape)

            # updating the velocity and position
            V_g[g] = next_v

            P_g[g] = next_p


            err_g = fitness_particles(X,g,P_g)

            PB_g, PE_g, gb_g, best_fit, min_idx = update_solution(P_g[g],PB_g,PE_g,gb_g,min_idx,err_g,best_fit)
        

        GB.append(gb_g)
    GB = np.stack(GB,0)

    return GB



In [10]:



import warnings
#warnings.filterwarnings('ignore')
np.random.seed(10)

global c1,c2,r0,r
r0 = np.random.uniform(0,1) # constant inertia weight (how much to weigh the previous velocity) in PSO and random in BAPSO
c1=2                        # cognative constant
c2=2                        # social constant
r = np.random.rand(2)       # r1,r2
maxit = 3000
ps = 70

for_all_data = []


n_exp = 4                           # total dataset
data = load_data(DATA,n_exp)        # load data 4 X 8 X 50
norm_data = data_normalize(data)    # normalize data

for d in range(0,norm_data.shape[0]):
    print("Data - {}".format(d))

    data0 = data[d]                     # (1st) one of 4 datasets [8 x 50]
    norm_data0 = norm_data[d]           # norm of that 1st


    N = norm_data0.shape[0]
    L = 10                              # No  of experiment for each generated GRN

    PS_ij = np.zeros((N,N))
    for l in range(0,L):

        print("Iteration-{}".format(l+1))
        GB = BAPSO(8,maxit,ps,data0) # Gb = 1x11

        f_ij = GB[:,:N] # NxN
        PS_ij = PS_ij + f_ij

    PS_ij = PS_ij/L


    # threshold
    ths = 0.9 #phi
    #print(PS_ij)
    PS_ij[PS_ij>ths] = 1
    PS_ij[PS_ij==ths] = 1
    PS_ij[PS_ij<ths] = 0

    g_ij = PS_ij

    for_all_data.append(g_ij)


#print(GB[:,:N])


Data - 0
Iteration-1


Gene 1:   0%|          | 0/2999 [00:00<?, ?it/s]C:\Users\INDRAN~1\AppData\Local\Temp/ipykernel_58604/1595326886.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  temp.append(pow(x_j_t[j],f_ij[j]))
C:\Users\INDRAN~1\AppData\Local\Temp/ipykernel_58604/1595326886.py:57: RuntimeWarning: invalid value encountered in double_scalars
  X_pred[g,t] = dt * (del_i * pi(X[:,t-1],f_ij)) + (1 - dt * sci_i) * X[g,t-1] + meu_i * d_i
C:\Users\INDRAN~1\AppData\Local\Temp/ipykernel_58604/1595326886.py:21: RuntimeWarning: invalid value encountered in double_scalars
  sum = sum + (X_i[t] - X_pred_i[t])
Gene 1:   9%|▉         | 283/2999 [00:15<02:29, 18.11it/s]


KeyboardInterrupt: 

In [5]:
saved_loc = 'C:\\Users\\Indrani Rana\\OneDrive\\Desktop\\project\\dada_half_systems\\result2_3000'
def save_matrices(matrix,saved_loc):
    '''
        matrix : an N number of 2-D matrices

    '''

    N = len(matrix)
    for i in range(0,N):
        name = 'data_{}'.format(i+1)+'.txt'
        print(i,name)
        loc = saved_loc + '/' + name

        np.savetxt(loc,matrix[i])

        print("File saved at {}".format(saved_loc))

# saved data level

save_matrices(for_all_data,saved_loc)
    


In [ ]:
#print(data)

def find_edges(f_ij):
    edges = []
    for g1 in range(0,f_ij.shape[0]):
        for g2 in range(0,f_ij.shape[1]):

            if f_ij[g1,g1] == 1:
                edges.append((gene_names[g1],gene_names[g2]))
    return edges


def plot(edges):


    import networkx as nx
    import matplotlib.pyplot as plt

    G = nx.MultiDiGraph
    G.add_edges_from(edges)
    #values = [val_map.get(node, 0.25) for node in G.nodes()]

    black_edges = [edge for edge in G.edges()]
    pos = nx.spring_layout(G)
    nx.draw_networkx_nodes(G, pos, cmap=plt.get_cmap('jet'), node_size = 500)
    nx.draw_networkx_labels(G, pos)
    nx.draw_networkx_edges(G, pos, edgelist=black_edges, arrows=False)
    plt.show()

edges = find_edges(for_all_data[3])
plot(edges)

In [6]:

import networkx as nx 
  
edges = [(1, 2), (1, 6), (2, 3), (2, 4), (2, 6),  
         (3, 4), (3, 5), (4, 8), (4, 9), (6, 7)] 

G = nx.MultiDiGraph() 
  
G.add_edges_from(edges) 
nx.draw_networkx(G, with_label = True)

ValueError: Received invalid argument(s): with_label

In [ ]:
# ['uvrD', 'lexA', 'umuDC', 'recA', 'uvrA', 'uvrY', 'ruvA', 'polB']
actual_edges = 

In [51]:
gene_names

['uvrD', 'lexA', 'umuDC', 'recA', 'uvrA', 'uvrY', 'ruvA', 'polB']